In [1]:
import pandas as pd
import numpy as np
import streamlit as st

In [2]:
brasileirao = pd.read_excel('../data/brasileirao2023.xlsx')
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
375,2023,38,GOI,NaN,AME,NaN
376,2023,38,SAN,NaN,FOR,NaN
377,2023,38,SAO,NaN,FLA,NaN
378,2023,38,VAS,NaN,RED,NaN


In [3]:
brasileirao['Score_m']

0      2.0
1      0.0
2      2.0
3      2.0
4      2.0
      ... 
375    NaN
376    NaN
377    NaN
378    NaN
379    NaN
Name: Score_m, Length: 380, dtype: float64

In [4]:
brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].count()

330

In [5]:
brasileirao = brasileirao[brasileirao['Score_m'].notnull()]
brasileirao

,Temporada,Rodada,Mandante,Score_m,Visitante,Score_v
0,2023,1,PAL,2.0,CUI,1.0
1,2023,1,AME,0.0,FLU,3.0
2,2023,1,CAP,2.0,GOI,0.0
3,2023,1,BOT,2.0,SAO,1.0
4,2023,1,RED,2.0,BAH,1.0
...,...,...,...,...,...,...
325,2023,33,CRU,0.0,VAS,0.0
326,2023,33,GOI,0.0,SAN,1.0
327,2023,33,COR,1.0,CAM,1.0
328,2023,33,BOT,3.0,GRE,4.0


In [6]:
brasileirao.dtypes

Temporada      int64
Rodada         int64
Mandante      object
Score_m      float64
Visitante     object
Score_v      float64
dtype: object

In [7]:
times = brasileirao['Mandante'].unique()
tabela = pd.DataFrame()
for time in times:
    vit = emp = der = pro = con = jog = 0
    # emp = 0
    # der = 0
    # pro = 0
    # con = 0
    # jog = 0
    mandante = brasileirao[brasileirao['Mandante'] == time]
    for indice, partida in mandante.iterrows():
        if partida['Score_m'] > partida['Score_v']:
            vit += 1 
        elif partida['Score_m'] == partida['Score_v']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_m']
        con += partida['Score_v']
        jog += 1
    visitante = brasileirao[brasileirao['Visitante'] == time]
    for indice, partida in visitante.iterrows():
        if partida['Score_v'] > partida['Score_m']:
            vit += 1
        elif partida['Score_v'] == partida['Score_m']:
            emp += 1
        else:
            der += 1
        pro += partida['Score_v']
        con += partida['Score_m']
        jog += 1
    new_row = pd.DataFrame({
        '1-Time': [time],
        '3-J' : [jog],
        '4-V' : [vit],
        '5-E' : [emp],
        '6-D' : [der],
        '7-GP': [pro],
        '8-GC': [con]
    })   
    tabela = pd.concat([tabela, new_row], ignore_index = True)
    
tabela['2-P'] = (tabela['4-V'] * 3) + tabela['5-E']
tabela['9-SG'] = tabela['7-GP'] - tabela['8-GC']   
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG
3,BOT,33,18,6,9,51.0,28.0,60,23.0
9,GRE,33,18,5,10,57.0,49.0,59,8.0
0,PAL,33,17,8,8,53.0,30.0,59,23.0
4,RED,33,16,11,6,44.0,27.0,59,17.0
8,FLA,33,16,9,8,49.0,36.0,57,13.0
6,CAM,33,15,9,9,41.0,26.0,54,15.0
2,CAP,33,13,11,9,46.0,38.0,50,8.0
10,FLU,33,13,8,12,42.0,41.0,47,1.0
12,SAO,33,12,10,11,37.0,35.0,46,2.0
5,FOR,33,12,9,12,35.0,35.0,45,0.0


In [8]:
tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]

C:\Users\User\AppData\Local\Temp\ipykernel_23144\1631230441.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tabela_sort[tabela_sort['7-GP'].notnull().sort_values(ascending=False)]['1-Time'].iloc[0]


'BOT'

In [9]:
a = brasileirao[brasileirao['Score_m'].notnull()]['Rodada'].unique().max()
a

33

In [10]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
# tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
# tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
# tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
# tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
# tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,33,17,8,8,53.0,30.0,59,23.0,1
1,AME,33,4,9,20,38.0,69.0,21,-31.0,3
2,CAP,33,13,11,9,46.0,38.0,50,8.0,4
3,BOT,33,18,6,9,51.0,28.0,60,23.0,1
4,RED,33,16,11,6,44.0,27.0,59,17.0,1
5,FOR,33,12,9,12,35.0,35.0,45,0.0,2
6,CAM,33,15,9,9,41.0,26.0,54,15.0,4
7,COR,33,9,14,10,38.0,39.0,41,-1.0,2
8,FLA,33,16,9,8,49.0,36.0,57,13.0,4
9,GRE,33,18,5,10,57.0,49.0,59,8.0,4


In [11]:
print(kmeans.labels_)

[1 3 4 1 1 2 4 2 4 4 2 2 2 2 2 0 0 3 0 0]


In [12]:
kmeans.cluster_centers_

array([[ 37.75      ,   9.75      ,   8.5       ,  14.75      ,
        -12.25      ],
       [ 59.33333333,  17.        ,   8.33333333,   7.66666667,
         21.        ],
       [ 43.57142857,  11.14285714,  10.14285714,  11.71428571,
          0.28571429],
       [ 23.5       ,   5.5       ,   7.        ,  20.5       ,
        -30.        ],
       [ 55.        ,  15.5       ,   8.5       ,   9.        ,
         11.        ]])

In [13]:
df_cluster = pd.DataFrame()
for cluster, colunas in enumerate(kmeans.cluster_centers_):
    print(cluster)
    print(colunas)
    new_row = pd.DataFrame({
        'cluster': [cluster], 
        '2-P': [colunas[0]],              
        '4-V': [colunas[1]],
        '5-E': [colunas[2]],
        '6-D': [colunas[3]],
        '9-SG': [colunas[4]],
    })
    df_cluster = pd.concat([df_cluster, new_row], ignore_index=True)
df_cluster

0
[ 37.75   9.75   8.5   14.75 -12.25]
1
[59.33333333 17.          8.33333333  7.66666667 21.        ]
2
[43.57142857 11.14285714 10.14285714 11.71428571  0.28571429]
3
[ 23.5   5.5   7.   20.5 -30. ]
4
[55.  15.5  8.5  9.  11. ]


,cluster,2-P,4-V,5-E,6-D,9-SG
0,0,37.750000,9.750000,8.500000,14.750000,-12.250000
1,1,59.333333,17.000000,8.333333,7.666667,21.000000
2,2,43.571429,11.142857,10.142857,11.714286,0.285714
3,3,23.500000,5.500000,7.000000,20.500000,-30.000000
4,4,55.000000,15.500000,8.500000,9.000000,11.000000


In [14]:
df_cluster_sort = df_cluster.sort_values(by='2-P', ascending=False)
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG
1,1,59.333333,17.000000,8.333333,7.666667,21.000000
4,4,55.000000,15.500000,8.500000,9.000000,11.000000
2,2,43.571429,11.142857,10.142857,11.714286,0.285714
0,0,37.750000,9.750000,8.500000,14.750000,-12.250000
3,3,23.500000,5.500000,7.000000,20.500000,-30.000000


In [15]:
df_cluster_sort['grupo'] = ['Título','Libertadores','Sul-Americana','Limbo','Rebaixamento']
df_cluster_sort

,cluster,2-P,4-V,5-E,6-D,9-SG,grupo
1,1,59.333333,17.000000,8.333333,7.666667,21.000000,Título
4,4,55.000000,15.500000,8.500000,9.000000,11.000000,Libertadores
2,2,43.571429,11.142857,10.142857,11.714286,0.285714,Sul-Americana
0,0,37.750000,9.750000,8.500000,14.750000,-12.250000,Limbo
3,3,23.500000,5.500000,7.000000,20.500000,-30.000000,Rebaixamento


In [16]:
df_cluster_grupo = df_cluster_sort[['cluster','grupo']]
df_cluster_grupo

,cluster,grupo
1,1,Título
4,4,Libertadores
2,2,Sul-Americana
0,0,Limbo
3,3,Rebaixamento


In [17]:
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
0,PAL,33,17,8,8,53.0,30.0,59,23.0,1
1,AME,33,4,9,20,38.0,69.0,21,-31.0,3
2,CAP,33,13,11,9,46.0,38.0,50,8.0,4
3,BOT,33,18,6,9,51.0,28.0,60,23.0,1
4,RED,33,16,11,6,44.0,27.0,59,17.0,1
5,FOR,33,12,9,12,35.0,35.0,45,0.0,2
6,CAM,33,15,9,9,41.0,26.0,54,15.0,4
7,COR,33,9,14,10,38.0,39.0,41,-1.0,2
8,FLA,33,16,9,8,49.0,36.0,57,13.0,4
9,GRE,33,18,5,10,57.0,49.0,59,8.0,4


In [18]:
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')
# tabela_sort.style.background_gradient(cmap='Blues').format("{:.2f}").render()


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster
3,BOT,33,18,6,9,51.000000,28.000000,60,23.000000,1
9,GRE,33,18,5,10,57.000000,49.000000,59,8.000000,4
0,PAL,33,17,8,8,53.000000,30.000000,59,23.000000,1
4,RED,33,16,11,6,44.000000,27.000000,59,17.000000,1
8,FLA,33,16,9,8,49.000000,36.000000,57,13.000000,4
6,CAM,33,15,9,9,41.000000,26.000000,54,15.000000,4
2,CAP,33,13,11,9,46.000000,38.000000,50,8.000000,4
10,FLU,33,13,8,12,42.000000,41.000000,47,1.000000,2
12,SAO,33,12,10,11,37.000000,35.000000,46,2.000000,2
5,FOR,33,12,9,12,35.000000,35.000000,45,0.000000,2


In [19]:
#NORMALIZAR A TABELA E COLORIR...

#grafico com pontuacao original e dos cluster

#grafico dos cluster

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [20]:
df_data = tabela[[
    'cluster',              
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG'
]]

In [21]:
X_Train = df_data.drop(columns=['cluster'], axis=1)
X_Test = df_data.drop(columns=['cluster'], axis=1)
y_Train = df_data['cluster']
y_Test = df_data['cluster']

In [22]:
sc_x = StandardScaler()
X_Train = sc_x.fit_transform(X_Train)
X_Test = sc_x.fit_transform(X_Test)

In [23]:
logreg = LogisticRegression(solver="lbfgs", max_iter=500)
logreg.fit(X_Train, y_Train)
pred_logreg = logreg.predict(X_Test)
pred_proba = logreg.predict_proba(X_Test)

In [24]:
tabela["cluster_pred"] = pred_logreg

In [25]:
lista_proba = pred_proba.tolist()

data = []  # Lista para armazenar os dicionários de dados

index = 0
for proba in lista_proba:
    for i in range(0, len(proba)):
        new_row = {"index": index, "prob": i, "valor": round(proba[i], 4)}
        data.append(new_row)
    index += 1

df_prob = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários
df_prob = df_prob.pivot_table(
    index="index", columns="prob", values="valor", aggfunc="sum"
)
df_prob = df_prob.reset_index()
df_prob = df_prob.set_index("index")


In [26]:
df_prob = df_prob.rename(columns={
    0.0: 'cl_o',
    1.0: 'cl_1',
    2.0: 'cl_2',
    3.0: 'cl_3',
    4.0: 'cl_4'
})

In [27]:
tabela = pd.merge(tabela, df_prob, left_index=True, right_index=True)
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,33,17,8,8,53.0,30.0,59,23.0,1,1,0.0050,0.5557,0.0786,0.0000,0.3607
1,AME,33,4,9,20,38.0,69.0,21,-31.0,3,3,0.2635,0.0000,0.0278,0.7086,0.0001
2,CAP,33,13,11,9,46.0,38.0,50,8.0,4,2,0.0505,0.1469,0.5005,0.0012,0.3010
3,BOT,33,18,6,9,51.0,28.0,60,23.0,1,1,0.0058,0.5637,0.0523,0.0001,0.3782
4,RED,33,16,11,6,44.0,27.0,59,17.0,1,1,0.0046,0.4725,0.1255,0.0000,0.3974
5,FOR,33,12,9,12,35.0,35.0,45,0.0,2,2,0.2200,0.0402,0.5503,0.0154,0.1741
6,CAM,33,15,9,9,41.0,26.0,54,15.0,4,4,0.0245,0.3255,0.2480,0.0004,0.4015
7,COR,33,9,14,10,38.0,39.0,41,-1.0,2,2,0.0913,0.0141,0.8264,0.0049,0.0633
8,FLA,33,16,9,8,49.0,36.0,57,13.0,4,4,0.0152,0.3682,0.1586,0.0002,0.4578
9,GRE,33,18,5,10,57.0,49.0,59,8.0,4,4,0.0320,0.3175,0.0778,0.0006,0.5722


In [28]:
df_pred = tabela[[
    '1-Time',
    'cl_o',
    'cl_1',
    'cl_2',
    'cl_3',
    'cl_4',
]]
df_pred = df_pred.copy()
df_pred

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.0050,0.5557,0.0786,0.0000,0.3607
1,AME,0.2635,0.0000,0.0278,0.7086,0.0001
2,CAP,0.0505,0.1469,0.5005,0.0012,0.3010
3,BOT,0.0058,0.5637,0.0523,0.0001,0.3782
4,RED,0.0046,0.4725,0.1255,0.0000,0.3974
5,FOR,0.2200,0.0402,0.5503,0.0154,0.1741
6,CAM,0.0245,0.3255,0.2480,0.0004,0.4015
7,COR,0.0913,0.0141,0.8264,0.0049,0.0633
8,FLA,0.0152,0.3682,0.1586,0.0002,0.4578
9,GRE,0.0320,0.3175,0.0778,0.0006,0.5722


In [29]:
df_pred.style.background_gradient(cmap='Blues', axis=1)

,1-Time,cl_o,cl_1,cl_2,cl_3,cl_4
0,PAL,0.005000,0.555700,0.078600,0.000000,0.360700
1,AME,0.263500,0.000000,0.027800,0.708600,0.000100
2,CAP,0.050500,0.146900,0.500500,0.001200,0.301000
3,BOT,0.005800,0.563700,0.052300,0.000100,0.378200
4,RED,0.004600,0.472500,0.125500,0.000000,0.397400
5,FOR,0.220000,0.040200,0.550300,0.015400,0.174100
6,CAM,0.024500,0.325500,0.248000,0.000400,0.401500
7,COR,0.091300,0.014100,0.826400,0.004900,0.063300
8,FLA,0.015200,0.368200,0.158600,0.000200,0.457800
9,GRE,0.032000,0.317500,0.077800,0.000600,0.572200


In [30]:
rodadas = brasileirao['Rodada'].unique()
times = brasileirao['Mandante'].unique()
pontuacao = pd.DataFrame()
data = []  # Lista para armazenar os dicionários de dados

for rodada in rodadas:
    for time in times:
        resultado = brasileirao[(brasileirao['Rodada'] == rodada) & ((brasileirao['Mandante'] == time) | (brasileirao['Visitante'] == time))]
        if len(resultado) > 0:
            resultado = resultado.reset_index()
            if resultado['Mandante'][0] == time:
                if resultado['Score_m'][0] > resultado['Score_v'][0]:
                    pontos = 3
                elif resultado['Score_m'][0] == resultado['Score_v'][0]:
                    pontos = 1
                else:
                    pontos = 0
            else:                  
                if resultado['Score_v'][0] > resultado['Score_m'][0]:
                    pontos = 3
                elif resultado['Score_v'][0] == resultado['Score_m'][0]:
                    pontos = 1
                else:
                    pontos = 0
            new_row = {
                'time': time,
                'rodada': rodada,
                'pontos': pontos
            }        
            data.append(new_row)  # Adiciona o dicionário à lista de dados

pontuacao = pd.DataFrame(data)  # Cria o DataFrame a partir da lista de dicionários

pt_pontuacao = pontuacao.pivot_table(index='rodada', columns='time', values='pontos', aggfunc='sum')
pt_pontuacao_cum = pt_pontuacao.cumsum()
pt_pontuacao_cum

time,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,FLA,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
rodada,,,,,,,,,,,,,,,,,,,,
1,0,0,3,0,3,0,3,0,0,3,3,1,0,3,1,3,3,0,0,3
2,0,0,6,1,3,0,3,3,1,3,6,4,3,3,4,4,4,1,3,4
3,0,3,9,4,3,1,3,6,1,3,6,7,3,6,7,7,4,4,4,4
4,0,6,12,4,6,1,4,9,4,3,7,8,3,7,7,10,5,4,7,5
5,1,6,15,7,9,2,4,9,4,6,10,9,3,7,7,13,6,7,8,6
6,1,6,15,10,12,2,5,12,4,9,13,10,6,8,7,14,7,10,9,6
7,4,7,18,13,12,2,5,12,7,12,13,10,7,11,7,15,10,11,12,6
8,4,7,21,14,12,3,8,13,8,13,13,13,7,14,10,16,13,11,15,6
9,7,8,21,17,15,3,8,13,11,16,16,14,7,17,11,19,13,12,15,6


In [31]:
pt_pontuacao_cum = pt_pontuacao_cum.reset_index()
pt_pontuacao_cum

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
0,1,0,0,3,0,3,0,3,0,0,...,3,1,0,3,1,3,3,0,0,3
1,2,0,0,6,1,3,0,3,3,1,...,6,4,3,3,4,4,4,1,3,4
2,3,0,3,9,4,3,1,3,6,1,...,6,7,3,6,7,7,4,4,4,4
3,4,0,6,12,4,6,1,4,9,4,...,7,8,3,7,7,10,5,4,7,5
4,5,1,6,15,7,9,2,4,9,4,...,10,9,3,7,7,13,6,7,8,6
5,6,1,6,15,10,12,2,5,12,4,...,13,10,6,8,7,14,7,10,9,6
6,7,4,7,18,13,12,2,5,12,7,...,13,10,7,11,7,15,10,11,12,6
7,8,4,7,21,14,12,3,8,13,8,...,13,13,7,14,10,16,13,11,15,6
8,9,7,8,21,17,15,3,8,13,11,...,16,14,7,17,11,19,13,12,15,6
9,10,8,9,24,18,16,4,9,14,12,...,17,14,8,17,14,22,14,13,15,6


In [32]:
from sklearn.linear_model import LinearRegression

In [33]:
colunas = pt_pontuacao_cum.columns
df_regressao = pd.DataFrame()
data_regressao = []  # Lista para armazenar os dicionários de resultados de regressão

for coluna in colunas:
    if coluna != 'rodada':
        X = pt_pontuacao_cum['rodada'].values.reshape(-1, 1)
        y = pt_pontuacao_cum[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        new_row_regressao = {
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        }
        data_regressao.append(new_row_regressao)

df_regressao = pd.DataFrame(data_regressao)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
2,BOT,6.31,1.87,77.48
16,RED,-2.59,1.86,68.09
15,PAL,2.52,1.68,66.32
9,FLA,-0.25,1.73,65.60
13,GRE,0.77,1.69,65.16
4,CAP,0.14,1.58,60.24
3,CAM,-1.29,1.61,59.78
10,FLU,3.07,1.44,57.71
11,FOR,1.34,1.42,55.18
18,SAO,2.32,1.28,50.98


In [34]:
df_regressao = df_regressao[['time','slope']]
df_regressao

,time,slope
0,AME,0.71
1,BAH,1.17
2,BOT,1.87
3,CAM,1.61
4,CAP,1.58
5,CFC,0.82
6,COR,1.29
7,CRU,1.11
8,CUI,1.39
9,FLA,1.73


In [35]:
tabela = tabela.merge(df_regressao, left_on='1-Time', right_on='time', how='left')
tabela

,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope
0,PAL,33,17,8,8,53.0,30.0,59,23.0,1,1,0.0050,0.5557,0.0786,0.0000,0.3607,PAL,1.68
1,AME,33,4,9,20,38.0,69.0,21,-31.0,3,3,0.2635,0.0000,0.0278,0.7086,0.0001,AME,0.71
2,CAP,33,13,11,9,46.0,38.0,50,8.0,4,2,0.0505,0.1469,0.5005,0.0012,0.3010,CAP,1.58
3,BOT,33,18,6,9,51.0,28.0,60,23.0,1,1,0.0058,0.5637,0.0523,0.0001,0.3782,BOT,1.87
4,RED,33,16,11,6,44.0,27.0,59,17.0,1,1,0.0046,0.4725,0.1255,0.0000,0.3974,RED,1.86
5,FOR,33,12,9,12,35.0,35.0,45,0.0,2,2,0.2200,0.0402,0.5503,0.0154,0.1741,FOR,1.42
6,CAM,33,15,9,9,41.0,26.0,54,15.0,4,4,0.0245,0.3255,0.2480,0.0004,0.4015,CAM,1.61
7,COR,33,9,14,10,38.0,39.0,41,-1.0,2,2,0.0913,0.0141,0.8264,0.0049,0.0633,COR,1.29
8,FLA,33,16,9,8,49.0,36.0,57,13.0,4,4,0.0152,0.3682,0.1586,0.0002,0.4578,FLA,1.73
9,GRE,33,18,5,10,57.0,49.0,59,8.0,4,4,0.0320,0.3175,0.0778,0.0006,0.5722,GRE,1.69


In [36]:
from sklearn.cluster import KMeans
df_data = tabela[[
    '2-P',              
    '4-V',
    '5-E',
    '6-D',
    '9-SG',
    'slope'
]]
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_data)
tabela['cluster'] = kmeans.labels_
tabela['c_p'] = kmeans.cluster_centers_[kmeans.labels_,0]
tabela['c_v'] = kmeans.cluster_centers_[kmeans.labels_,1]
tabela['c_e'] = kmeans.cluster_centers_[kmeans.labels_,2]
tabela['c_d'] = kmeans.cluster_centers_[kmeans.labels_,3]
tabela['c_sg'] = kmeans.cluster_centers_[kmeans.labels_,4]
tabela['c_slope'] = kmeans.cluster_centers_[kmeans.labels_,5]
tabela_sort = tabela.sort_values(by=['2-P','4-V','9-SG'], ascending=False)
tabela_sort.style.background_gradient(cmap='Blues')


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,1-Time,3-J,4-V,5-E,6-D,7-GP,8-GC,2-P,9-SG,cluster,cluster_pred,cl_o,cl_1,cl_2,cl_3,cl_4,time,slope,c_p,c_v,c_e,c_d,c_sg,c_slope
3,BOT,33,18,6,9,51.000000,28.000000,60,23.000000,1,1,0.005800,0.563700,0.052300,0.000100,0.378200,BOT,1.870000,59.333333,17.000000,8.333333,7.666667,21.000000,1.803333
9,GRE,33,18,5,10,57.000000,49.000000,59,8.000000,4,4,0.032000,0.317500,0.077800,0.000600,0.572200,GRE,1.690000,55.000000,15.500000,8.500000,9.000000,11.000000,1.652500
0,PAL,33,17,8,8,53.000000,30.000000,59,23.000000,1,1,0.005000,0.555700,0.078600,0.000000,0.360700,PAL,1.680000,59.333333,17.000000,8.333333,7.666667,21.000000,1.803333
4,RED,33,16,11,6,44.000000,27.000000,59,17.000000,1,1,0.004600,0.472500,0.125500,0.000000,0.397400,RED,1.860000,59.333333,17.000000,8.333333,7.666667,21.000000,1.803333
8,FLA,33,16,9,8,49.000000,36.000000,57,13.000000,4,4,0.015200,0.368200,0.158600,0.000200,0.457800,FLA,1.730000,55.000000,15.500000,8.500000,9.000000,11.000000,1.652500
6,CAM,33,15,9,9,41.000000,26.000000,54,15.000000,4,4,0.024500,0.325500,0.248000,0.000400,0.401500,CAM,1.610000,55.000000,15.500000,8.500000,9.000000,11.000000,1.652500
2,CAP,33,13,11,9,46.000000,38.000000,50,8.000000,4,2,0.050500,0.146900,0.500500,0.001200,0.301000,CAP,1.580000,55.000000,15.500000,8.500000,9.000000,11.000000,1.652500
10,FLU,33,13,8,12,42.000000,41.000000,47,1.000000,2,2,0.215100,0.062400,0.464900,0.013400,0.244300,FLU,1.440000,43.571429,11.142857,10.142857,11.714286,0.285714,1.304286
12,SAO,33,12,10,11,37.000000,35.000000,46,2.000000,2,2,0.146000,0.055300,0.594300,0.007400,0.197000,SAO,1.280000,43.571429,11.142857,10.142857,11.714286,0.285714,1.304286
5,FOR,33,12,9,12,35.000000,35.000000,45,0.000000,2,2,0.220000,0.040200,0.550300,0.015400,0.174100,FOR,1.420000,43.571429,11.142857,10.142857,11.714286,0.285714,1.304286


In [37]:
pontuacao_2 = pt_pontuacao_cum[pt_pontuacao_cum['rodada'] > 19]
pontuacao_2

time,rodada,AME,BAH,BOT,CAM,CAP,CFC,COR,CRU,CUI,...,FLU,FOR,GOI,GRE,INT,PAL,RED,SAN,SAO,VAS
19,20,10,21,48,27,32,14,25,25,28,...,34,29,23,34,24,37,32,21,28,19
20,21,13,21,51,30,33,14,26,25,28,...,35,32,24,37,25,40,35,21,28,19
21,22,16,22,51,31,34,14,27,26,28,...,38,32,25,40,26,41,36,21,31,20
22,23,17,25,51,34,37,14,27,29,29,...,38,35,25,40,29,44,39,21,31,23
23,24,17,25,51,37,40,14,30,29,29,...,41,38,26,43,29,44,42,24,31,26
24,25,18,25,52,40,40,17,30,30,32,...,41,39,27,44,29,44,45,27,34,26
25,26,18,28,55,40,41,20,31,31,33,...,41,42,27,44,32,44,46,30,35,27
26,27,18,31,58,43,44,20,32,31,36,...,42,42,30,44,32,44,49,30,35,30
27,28,19,34,59,43,45,20,33,34,37,...,42,42,31,44,35,47,52,30,38,30
28,29,19,34,60,46,48,20,36,37,37,...,45,43,31,47,38,50,52,33,38,30


In [38]:
colunas = pontuacao_2.columns
df_regressao = pd.DataFrame()
colunas = pontuacao_2.columns
regressao_data = []

for coluna in colunas:
    if coluna != 'rodada':
        X = pontuacao_2['rodada'].values.reshape(-1, 1)
        y = pontuacao_2[coluna].values.reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(X, y)
        A = regressor.intercept_[0]
        B = regressor.coef_[0][0]
        x = A + (B * 38)
        regressao_data.append({
            'time': coluna,
            'intercept': round(A, 2),
            'slope': round(B, 2),
            'pontuacao_final': round(x, 2)
        })

df_regressao = pd.DataFrame(regressao_data)
df_regressao.sort_values(by='pontuacao_final', ascending=False)

,time,intercept,slope,pontuacao_final
16,RED,-9.53,2.13,71.23
2,BOT,28.17,1.03,67.26
15,PAL,4.65,1.61,65.79
3,CAM,-14.26,2.10,65.49
9,FLA,1.75,1.65,64.38
13,GRE,2.20,1.63,64.09
4,CAP,2.20,1.51,59.58
11,FOR,7.62,1.20,53.13
10,FLU,16.62,0.94,52.20
18,SAO,0.64,1.32,50.83
